<a href="https://colab.research.google.com/github/AlexKI123/DoublesProject/blob/main/KerasTuner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install keras-tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.9/128.9 kB 4.9 MB/s eta 0:00:00


In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import keras_tuner as kt

# Load the data
train_data = pd.read_csv('/content/drive/MyDrive/input_training.csv', index_col=0)
train_data_y = pd.read_csv('/content/drive/MyDrive/output_training_gmEd6Zt.csv', index_col=0)

# Merge the input and output data on the 'ID' column
merged_data = train_data.merge(train_data_y, left_index=True, right_index=True)

# Scale the training data
division_value = 100
columns_to_transform = [f'r{i}' for i in range(53)]
for column in columns_to_transform:
    merged_data[column] = merged_data[column] / division_value
merged_data[columns_to_transform] = np.tanh(merged_data[columns_to_transform])

# Replace NaN values with the mask value
mask_value = -2.0
merged_data.fillna(mask_value, inplace=True)

# Drop the 'day' and 'equity' columns from the DataFrame
merged_data = merged_data.drop(['day', 'equity'], axis=1)

# Map labels from [-1, 0, 1] to [0, 1, 2]
label_mapping = {-1: 0, 0: 1, 1: 2}
merged_data['reod'] = merged_data['reod'].map(label_mapping)

# Prepare the data
X = merged_data.drop('reod', axis=1).values.reshape(-1, 53, 1)
y = to_categorical(merged_data['reod'].values)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

def build_model(hp):
    model = Sequential()
    model.add(LSTM(units=hp.Int('units_lstm_1', min_value=60, max_value=140, step=20), return_sequences=True, input_shape=(53, 1)))
    model.add(LSTM(units=hp.Int('units_lstm_2', min_value=30, max_value=80, step=20), return_sequences=False))
    model.add(Dropout(rate=hp.Float('dropout', min_value=0.0, max_value=0.5, step=0.05)))
    model.add(Dense(units=hp.Int('dense_units', min_value=32, max_value=96, step=32), activation='relu'))
    model.add(Dense(3, activation='softmax'))
    model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

tuner = kt.Hyperband(build_model, objective='val_accuracy', max_epochs=20, directory='keras_tuner_dir', project_name='lstm_optimization')

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, min_lr=0.00001)
#early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Execute the search
tuner.search(X_train, y_train, epochs=20, validation_split=0.2, callbacks=[reduce_lr])

# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The optimal number of units in the first LSTM layer is {best_hps.get('units_lstm_1')},
the optimal number of units in the second LSTM layer is {best_hps.get('units_lstm_2')},
the optimal dropout rate is {best_hps.get('dropout')}, and
the optimal number of neurons in the first dense layer is {best_hps.get('dense_units')}.
""")

# Build the model with the optimal hyperparameters and train it on the data
model = tuner.hypermodel.build(best_hps)
history = model.fit(X_train, y_train, epochs=20, validation_split=0.1, callbacks=[reduce_lr])

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f'Test loss: {loss}, Test accuracy: {accuracy}')


Trial 30 Complete [01h 05m 12s]
val_accuracy: 0.4776539206504822

Best val_accuracy So Far: 0.4837740659713745
Total elapsed time: 12h 30m 50s

The optimal number of units in the first LSTM layer is 100,
the optimal number of units in the second LSTM layer is 50,
the optimal dropout rate is 0.1, and
the optimal number of neurons in the first dense layer is 96.

Epoch 1/20
21346/21346 [==============================] - 202s 9ms/step - loss: 1.0219 - accuracy: 0.4449 - val_loss: 1.0063 - val_accuracy: 0.4636 - lr: 0.0010
Epoch 2/20
21346/21346 [==============================] - 197s 9ms/step - loss: 1.0025 - accuracy: 0.4690 - val_loss: 1.0016 - val_accuracy: 0.4701 - lr: 0.0010
Epoch 3/20
21346/21346 [==============================] - 198s 9ms/step - loss: 1.0011 - accuracy: 0.4711 - val_loss: 1.0002 - val_accuracy: 0.4719 - lr: 0.0010
Epoch 4/20
21346/21346 [==============================] - 197s 9ms/step - loss: 0.9994 - accuracy: 0.4740 - val_loss: 0.9998 - val_accuracy: 0.4742 - lr: